In [0]:
%sql
SELECT * FROM dbxmetagen.metadata_results.pi_metadata_generation_log ORDER BY _created_at DESC;

In [0]:
%sql
SELECT * FROM dbxmetagen.metadata_results.table_processing_log ORDER BY `_updated_at` DESC;

In [0]:
# Create the Delta table with room_number as a primary key
spark.sql("""
CREATE OR REPLACE TABLE dbxmetagen.default.rooms (
    room_number STRING,
    room_name STRING,
    capacity INT,
    PRIMARY KEY (room_number)
) USING DELTA
""")

# Insert 3 rows into the rooms table
spark.sql("""
INSERT INTO dbxmetagen.default.rooms (room_number, room_name, capacity) VALUES
('101A', 'Room A', 2),
('102', 'Room B', 4),
('103', 'Room C', 3)
""")

# First, alter the column to be non-nullable
try:
    spark.sql("""
    ALTER TABLE dbxmetagen.default.healthcare_test
    ALTER COLUMN patient_id SET NOT NULL
    """)

    # Then, add the primary key constraint
    spark.sql("""
    ALTER TABLE dbxmetagen.default.healthcare_test
    ADD CONSTRAINT patient_pk PRIMARY KEY (patient_id)
    """)
except: 
    print("already added or didnt work")

# Finally, add the foreign key constraint
spark.sql("""
ALTER TABLE dbxmetagen.default.healthcare_test
ADD CONSTRAINT room_number_fk
FOREIGN KEY (room_number) REFERENCES dbxmetagen.default.rooms
""")